# Proceso de entrenamiento de Zephyr 7B Beta
Este proceso de entrenamiento ha sido realizado con Google Colab PRO, en el entorno de ejecución llamado A100 GPU, el cual contiene GPUs con la funcionalidad CUDA. Si no se cuenta con este entorno, es posible que este notebook no funcione con el entorno que tengas actualmente.

## Librerías
Primero, es necesario instalar las librerías y paquetes necesarios para este proceso:
*   **transformers**: Proporciona herramientas para cargar, configurar y trabajar con modelos de procesamiento de lenguaje, facilitando tanto la inferencia como el entrenamiento.
*   **peft**: Sus siglas significan Parameter Efficient Fine-Tuning, y tal como su nombre indica, permite realizar fine-tuning eficiente en modelos grandes con técnicas como LoRA (Low-Rank Adaptation). Es clave para ajustar modelos grandes reduciendo el consumo de memoria y los tiempos de cómputo.
*   **torch**: Es el marco fundamental para el cálculo numérico y las redes neuronales en este entorno. Proporciona soporte para realizar operaciones tensoriales en la GPU.
*   **bitsandbytes**: Es una biblioteca especializada en optimizar el manejo de modelos en formatos de precisión reducida (como 4-bit y 8-bit), permitiendo que los modelos sean más ligeros y manejables.
*   **accelerate**: Proporciona herramientas para facilitar la ejecución en múltiples dispositivos (GPUs, TPUs, etc.) y optimizar la capacitación distribuida. Simplifica la configuración de modelos grandes para aprovechar al máximo los recursos de hardware.
*   **huggingface_hub**: Proporciona acceso al repositorio de modelos y datasets de Hugging Face mediante un token personal para autenticarse. Se utiliza para descargar modelos preentrenados y para guardar o compartir modelos ajustados.
*   **trl**: Es un paquete necesario para poder acceder a SFTTrainer (Supervised Fine-Tuning Trainer), una herramienta que simplifica el entrenamiento supervisado de modelos grandes, integrando directamente configuraciones como LoRA, datasets personalizados y optimización de hiperparámetros.






In [ ]:
!pip install transformers peft torch bitsandbytes accelerate huggingface_hub trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0


## Iniciar sesión en Hugging-Face
Este paso no es obligatorio, ya que principalmente sirve para subir el modelo fine-tuneado a tu repositorio. En caso de hacerlo, es necesario configurar un secret, ya que escribir el token y subir el código público, puede significar una brecha de seguridad para tu cuenta, ya que cualquiera podría acceder a ella con tu token.

Para hacerlo, en el panel situado a la izquierda de la pantalla, habrá un icono de una llave. Al darle click, podemos darle a "Añadir nuevo secreto", marcar la opción de "Acceso desde el cuaderno", llamarlo hf_token y asignarle el valor de nuestro token.

In [ ]:
from google.colab import userdata
hf_token = userdata.get('hf_token')
!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `All App` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `All App`


## Cargar el modelo
Aquí, organizamos las librerías que vamos a necesitar para el proceso de entrenamiento, y cargamos el modelo Zephyr 7B Beta, configurando unos parámetros para que el modelo se cargue de manera eficiente, optimizando el uso de memoria y computación del hardware disponible,a la vez que se adapta a las necesidades del fine-tuning.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_name = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True,                                 # Carga el modelo en un formato de 4 bits para reducir su tamaño y consumo de memoria
                                           bnb_4bit_compute_dtype=getattr(torch, "float16"),  #Define que los cálculos se realizarán en precisión de 16 bits para equilibrar precisión y eficiencia
                                           bnb_4bit_quant_type="nf4"),                        # Especifica el tipo de cuantización "nf4" que sirve para mejorar la estabilidad numérica y la precisión en comparación con los esquemas de cuantización estándar
    torch_dtype=torch.bfloat16, # Define que el modelo usará el tipo de dato bfloat16 (un formato más eficiente que FP32) para manejar tensores. bfloat16 es útil para modelos grandes porque reduce el uso de memoria mientras mantiene suficiente precisión
    device_map="auto",          # Permite que Hugging Face gestione automáticamente la asignación de las distintas partes del modelo entre dispositivos disponibles (por ejemplo, múltiples GPUs)
    trust_remote_code=True,     # Habilita la descarga de código específico para el modelo directamente desde Hugging Face Hub
)

Loading checkpoint shards:   100%|          | 8/8 [00:06<00:00, 1.28it/s]

## Definición de los hiperparámetros para el entrenamiento
Esta celda configura y prepara tanto el modelo como el tokenizador para el proceso de fine-tuning. Se ajusta el modelo para entrenamiento eficiente, se define la configuración de entrenamiento y se ajustan los hiperparámetros relevantes. Aquí se establece el entorno necesario para realizar un ajuste fino supervisado con un enfoque eficiente en recursos.

In [ ]:
model.config.use_cache = False          # Desactiva el uso de caché, que es útil para inferencia pero innecesario en entrenamiento.
model.config.pretraining_tp = 1         # Ajusta la cantidad de tensor parallelism usada para preentrenamiento, configurándolo para una sola división.
model.gradient_checkpointing_disable()  # Desactiva el almacenamiento de gradientes intermedios para ahorrar memoria, ya que no se utiliza en este caso.



tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Carga el tokenizador preentrenado del modelo Zephyr 7B desde Hugging Face Hub.
tokenizer.padding_side = 'right'                                              # Configura el padding del tokenizador en el lado derecho de las entradas.
tokenizer.pad_token = tokenizer.eos_token                                     # Define el token de padding como el token de fin de secuencia (EOS).

tokenizer.add_eos_token = True                                                # Asegura que las secuencias se completen con un token EOS.
tokenizer.add_bos_token, tokenizer.add_eos_token                              # Configura que se pueda incluir BOS (inicio de secuencia) y EOS (fin de secuencia).

model = prepare_model_for_kbit_training(model) # Ajusta el modelo para entrenar usando cuantización en bits más bajos para optimizar la memoria y el uso de hardware.

lora_config = LoraConfig(
    r=16,                                 # Dimensión interna de las matrices LoRA. Un valor más alto permite que el modelo capture patrones más complejos, pero aumenta el uso de memoria y cómputo.
    lora_alpha=16,                        # Escala de las actualizaciones de LoRA. Un valor más alto amplifica las actualizaciones, lo que puede ser útil para ajustar modelos grandes con datos complejos, pero también podría llevar a un sobreajuste.
    lora_dropout=0.1,                     # Tasa de dropout para regularización durante entrenamiento. Un valor más alto reduce el riesgo de sobreajuste, pero puede dificultar que el modelo aprenda patrones específicos.
    target_modules=["q_proj", "v_proj"],  # Aplica LoRA solo a módulos seleccionados (proyecciones de consulta y valores del modelo).
    task_type="CAUSAL_LM"                 # Especifica que el modelo es para tareas de lenguaje causal.
)

model = get_peft_model(model, lora_config) # Integra la configuración LoRA en el modelo

training_args = TrainingArguments(
    output_dir="./results",                # Directorio donde se guardarán los resultados y checkpoints.
    save_steps=500,                        # Guarda un checkpoint cada 500 pasos de entrenamiento.
    logging_steps=50,                      # Muestra logs de progreso cada 50 pasos.
    eval_strategy="steps",                 # Realiza evaluación periódica basada en pasos.
    per_device_train_batch_size=8,         # Tamaño del batch por dispositivo (GPU).
    gradient_accumulation_steps=1,         # Pasos de acumulación de gradientes antes de actualización.
    learning_rate=2e-5,                    # Tasa de aprendizaje inicial para el optimizador.
    weight_decay=0.01,                     # Decaimiento del peso (regularización).
    optim="paged_adamw_32bit",             # Optimizador AdamW paginado con precisión de 32 bits.
    num_train_epochs=2,                    # Número de épocas completas para entrenamiento.
    save_total_limit=3,                    # Mantiene un máximo de 3 checkpoints guardados.
    fp16=False,                            # Deshabilita precisión mixta de 16 bits (no requerida aquí).
    bf16=True,                             # Habilita precisión mixta de tipo bfloat16 (recomendado en GPUs modernas). !! Cambiar este parametro a false y fp16 a true en caso de no utilizar Google Colab PRO.
    max_grad_norm=1.0,                     # Límite de norma para los gradientes, evitando explosiones de gradientes.
    max_steps=-1,                          # Ignorado, ya que el número de pasos lo controla `num_train_epochs`.
    warmup_ratio=0.1,                      # Proporción del calentamiento para el optimizador (10% del total).
    group_by_length=True,                  # Agrupa datos por longitud para mejorar la eficiencia de memoria.
    lr_scheduler_type="linear",            # Usa un decaimiento lineal para la tasa de aprendizaje.
    report_to="wandb"                      # Reporta métricas de entrenamiento al sistema de monitoreo Weights & Biases (wandb).
)

## Procesamiento, formateo y preparación de los datos
Esta celda realiza el procesamiento, formateo y preparación de los datasets necesarios para el entrenamiento y evaluación del modelo.

In [ ]:
from datasets import Dataset, load_dataset, concatenate_datasets
from transformers import DataCollatorForLanguageModeling
import json
import os

# Función para formatear un dataset general
def format_general_dataset(sample):
    """Preserva la estructura del dataset sin agregar redundancias."""
    try:
        system_prompt = sample["<|system|>"]
        user_prompt = sample["<|user|>"]
        assistant_response = sample["<|assistant|>"]

        # Reestructurar para una clave unificada sin alterar el contenido
        sample["text"] = f"<|system|>: {system_prompt}\n<|user|>: {user_prompt}\n<|assistant|>: {assistant_response}"
    except KeyError as e:
        raise ValueError(f"Falta una clave requerida en el dataset: {e}")

    return sample

# Función para formatear un dataset de recomendaciones
def format_recommendations_dataset(sample):
    """Formatea el dataset inicial de recomendaciones."""
    try:
        system_prompt = sample["<|system|>"]
        user_data = json.loads(sample["<|user|>"])["user_profile"]
        assistant_response = sample["<|assistant|>"]

        # Formatear el perfil del usuario en texto claro
        user_profile = (
            f"User Profile:\n"
            f"- Age: {user_data.get('edad')}\n"
            f"- Monthly Income: {user_data.get('ingresos_mensuales')}\n"
            f"- Monthly Savings: {user_data.get('ahorro_mensual')}\n"
            f"- Risk Tolerance: {user_data.get('tolerancia_riesgo')}\n"
            f"- Investment Horizon: {user_data.get('horizonte_inversion')}\n"
            f"- Financial Goal: {user_data.get('objetivo_financiero')}\n"
        )

        # Generar texto formateado para el modelo
        sample["text"] = f"<|system|>: {system_prompt}\n<|user|>: {user_profile}\n<|assistant|>: {assistant_response}"
    except (KeyError, json.JSONDecodeError) as e:
        raise ValueError(f"Error al procesar el dataset `recomendaciones_iniciales`: {e}")

    return sample

# Definición de archivos de datasets
datasets = {
    "comparaciones_esquematico": os.path.join("q&a_comparaciones_esquematico.jsonl"),
    "recomendaciones_iniciales": os.path.join("recomendaciones_iniciales.jsonl"),
    "comparaciones_conversacionales": os.path.join("q&a_comparaciones_conversacionales.jsonl"),
    "conceptos_inversiones": os.path.join("conceptos_inversiones_Q&A.jsonl")
}

# Procesar datasets con el formateo correspondiente
processed_datasets = {}
for name, path in datasets.items():
    dataset = load_dataset("json", data_files=path)["train"]
    if name == "recomendaciones_iniciales":
        formatted_dataset = dataset.map(format_recommendations_dataset, remove_columns=["<|system|>", "<|user|>", "<|assistant|>"])
    else:
        formatted_dataset = dataset.map(format_general_dataset, remove_columns=["<|system|>", "<|user|>", "<|assistant|>"])

    processed_datasets[name] = formatted_dataset

# Combinar datasets
combined_dataset = concatenate_datasets([processed_datasets[name] for name in processed_datasets])

# Dividir en entrenamiento y evaluación
final_dataset = combined_dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = final_dataset["train"]
eval_dataset = final_dataset["test"]

# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False porque es un modelo causal, no de modelado de lenguaje enmascarado
)

print(f"Conjunto de entrenamiento: {len(train_dataset)} muestras")
print(f"Conjunto de evaluación: {len(eval_dataset)} muestras")


Conjunto de entrenamiento: 2859 muestras
Conjunto de evaluación: 505 muestras


## Configuración del trainer
Este bloque configura el trainer para realizar el fine-tune supervisado del modelo usando SFTTrainer. Este entrenador simplifica el proceso de entrenamiento supervisado, integrando configuraciones específicas como LoRA para reducir los requisitos de memoria.

In [ ]:
from trl import SFTTrainer
from accelerate import Accelerator

trainer = SFTTrainer(
    model = model,                     # El modelo cargado y configurado para ajuste fino (incluye LoRA y cuantización).
    args = training_args,              # Argumentos de configuración del entrenamiento definidos previamente.
    train_dataset = train_dataset,     # Dataset de entrenamiento
    eval_dataset = eval_dataset,       # Dataset de evaluación para validar el desempeño del modelo durante el entrenamiento.
    peft_config = lora_config,         # Configuración de LoRA para ajuste fino eficiente.
    processing_class = tokenizer,      # Tokenizador asociado al modelo, responsable de preparar los textos de entrada.
    data_collator = data_collator,     # Collator para agrupar datos, asegurando que tengan formato adecuado para el modelo.
)

## Configuración de WandB
Este bloque configura Weights & Biases (WandB), que es una herramienta de seguimiento y visualización de experimentos de machine learning. Primero se autentica con una clave de acceso (wandb.login), y luego inicializa un nuevo experimento (wandb.init) con un nombre de proyecto (Fine tuning Zephyr 7B), tipo de tarea (training), y permite que los datos se registren de forma anónima. Esto permite monitorear métricas como pérdida, precisión y otros detalles del entrenamiento en tiempo real.

In [ ]:
import wandb
wandb_token = userdata.get('wandb_token')
wandb.login(key = wandb_token)
run = wandb.init(
    project='Fine tuning Zephyr 7B',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ebayego (ebayego-universitat-oberta-de-catalunya). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Entrenamiento del modelo
A continuación, se entrena el modelo con todos los parámetros y datos definidos en los bloques anteriores.

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,1.635000,1.496082
100,1.286700,0.987415
150,0.734800,0.517745
200,0.424300,0.370512
250,0.312300,0.276465
300,0.258800,0.239283
350,0.231300,0.216503
400,0.170100,0.200281
450,0.182800,0.193877
500,0.190400,0.186046


TrainOutput(global_step=716, training_loss=0.43346010439888727, metrics={'train_runtime': 2388.8506, 'train_samples_per_second': 2.394, 'train_steps_per_second': 0.3, 'total_flos': 1.9163662773178368e+17, 'train_loss': 0.43346010439888727, 'epoch': 2.0})

## Guardar el modelo
Una vez ha finalizado el entrenamiento, hay que guardar el modelo para poder utilizarlo en futuras ocasiones. Este guardado, al utilizar el modelo el enfoque de LoRA que realiza ajustes en capas específicas del modelo base, al guardar el modelo, por defecto, solo se almacenan los parámetros de las capas ajustadas (el adaptador LoRA).

Además, con la función 'trainer.model.push_to_hub' subimos el modelo a nuestro repositorio de Hugging Face.

In [ ]:
new_model = "TFG"
trainer.model.save_pretrained(new_model)
wandb.finish()

trainer.model.push_to_hub(new_model, use_temp_dir=False)

print("Fine-tuning completado. Modelo y tokenizador guardados en './final_model'.")

eval/loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁
eval/runtime,▆▇▃▃▆▇▇█▄▇▁▄▄▅
eval/samples_per_second,▃▂▆▆▃▂▂▁▅▂█▅▅▄
eval/steps_per_second,▃▂▆▆▂▂▂▁▅▂█▅▅▄
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/grad_norm,▄▄▅▄▄▄█▁▂▂▂▂▂▂
train/learning_rate,▆█▇▇▆▆▅▄▄▃▃▂▂▁
train/loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁
eval/loss,0.17895
eval/runtime,47.5255


adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Fine-tuning completado. Modelo y tokenizador guardados en './final_model'.


## Guardado completo del modelo
También es posible guardar todas las capas del modelo en vez de solo las modificadas con LoRA, ya que al importar el modelo nuevamente para realizarle inferencia, será necesario primero descargar el modelo base y luego aplicarle las capas modificadas guardadas en el bloque anterior. Guardando el modelo de esta forma, nos ahorramos el paso anterior.

In [ ]:
model = model.merge_and_unload()

# Guardar el modelo completo
output_dir = "./final_full_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Modelo completo guardado en '{output_dir}'.")

Modelo completo guardado en './final_full_model'.


## Inferencia del modelo
Este bloque crea un pipeline de generación de texto usando la biblioteca transformers. El pipeline es una interfaz simplificada para generar texto con el modelo fine-tuneado. Los parámetros controlan aspectos como la longitud de las respuestas, la aleatoriedad y la diversidad de las generaciones, adaptándolo a tareas como responder preguntas o generar texto coherente.

In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",         # Especifica la tarea de generación de texto.
    model=model,                    # Modelo fine-tuneado que se usará para generar texto.
    tokenizer=tokenizer,            # Tokenizador asociado al modelo, para procesar las entradas y salidas.
    max_length=1000,                # Longitud máxima del texto generado para evitar respuestas excesivamente largas.
    min_length=100                  # Longitud mínima para asegurar que las respuestas no se trunquen demasiado.
    temperature=0.9,                # Controla la aleatoriedad del texto generado. Valores altos (cercanos a 1) producen respuestas más variadas.
    top_p=0.95,                     # Ajusta la diversidad de las respuestas usando "nucleus sampling". Limita las palabras generadas a un subconjunto con el 95% de probabilidad acumulada.
    eos_token_id=tokenizer.eos_token_id # Detiene la generación al encontrar el token de fin de secuencia (EOS).
)

Device set to use cuda:0


## Pruebas de preguntas
Los siguientes bloques, contienen prompts de ejemplo y el output que calcula el modelo para cada uno. Estas respuestas, ya se puede observar que utilizan parte del conocimiento adquirido gracias al fine-tuning.

In [ ]:
prompt = "¿Cuál es la diferencia entre ETFs de acumulación y distribución?"
result = pipe(prompt)
result_text = result[0]['generated_text']
cleaned_text = result_text.split("<|assistant|>")[-1].strip()
print(cleaned_text)

Las ETFs (Exchange Traded Funds) se pueden clasificar en dos tipos principales: ETFs de acumulación y ETFs de distribución. La principal diferencia entre ambos es cómo se distribuyen los ingresos y los gastos entre los accionistas.

En los ETFs de acumulación, los ingresos y los gastos se acumulan en la cartera de la ETF, y se distribuyen a los accionistas en forma de capital de distribución al final del año fiscal. Esto significa que los accionistas pueden optar por reinvertir los ingresos en la ETF o retirirlos en forma de dividendos. Los ETFs de acumulación pueden ser atractivos para los inversores que buscan un rendimiento más estable y predictible, ya que los ingresos se acumulan y se distribuyen en forma de capital de distribución, lo que puede reducir la volatilidad de los ingresos.

En los ETFs de distribución, los ingresos y los gastos se distribuyen a los accionistas en forma de dividendos mensuales o trimestrales. Esto significa que los accionistas pueden optar por reinverti

In [ ]:
prompt = "¿Cuales son las principales diferencias entre el MSCI World y el SP&500?"
result = pipe(prompt)
print(result[0]['generated_text'].replace("<|assistant|>", "").strip())

¿Cuales son las principales diferencias entre el MSCI World y el SP&500? 

Aunque ambos índices son medidas de rendimiento de la bolsa, hay algunas principales diferencias entre el MSCI World y el SP&500:

1. Composición: El MSCI World es un índice global que mide el desempeño de las principales empresas de 23 países desarrollados y emergentes, mientras que el SP&500 se centra en las empresas más grandes y líderes de la economía estadounidense.

2. Países: El MSCI World incluye empresas de países como Alemania, Japón, Reino Unido, Francia, China, Hong Kong, Taiwán, Sudáfrica, Australia, Nueva Zelanda, Singapur, Taiwán, Corea del Sur, y otros, además de los Estados Unidos. El SP&500 solo incluye empresas de los Estados Unidos.

3. Sectorial: El MSCI World tiene una distribución sectorial más diversa que el SP&500, con una mayor representación de empresas de la industria tecnológica, de la salud y de la energía renovable. El SP&500 tiene una mayor representación de empresas de la industr

In [ ]:
prompt = ("<|system|>: Eres un asistente financiero personal especializado en inversiones. "
          "Tu objetivo es ayudar a los usuarios con cualquier tema relacionado con inversiones, finanzas o economía. "
          "Para cada respuesta:\n"
          "1. Primero, busca los datos adecuados de tus conocimientos aprendidos o de las bases de datos externas disponibles.\n"
          "2. Luego, analiza cuidadosamente la consulta para dar una respuesta clara, precisa y personalizada.\n"
          "3. Mantente amable, profesional y enfocado únicamente en temas relacionados con inversiones, finanzas y economía.\n"
          "<|user|>: ¿En qué aspectos son diferentes UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C?"
          "<|assistant|>:"
)
result = pipe(prompt)
print(result[0]['generated_text'])

<|system|>: Eres un asistente financiero personal especializado en inversiones. Tu objetivo es ayudar a los usuarios con cualquier tema relacionado con inversiones, finanzas o economía. Para cada respuesta:
1. Primero, busca los datos adecuados de tus conocimientos aprendidos o de las bases de datos externas disponibles.
2. Luego, analiza cuidadosamente la consulta para dar una respuesta clara, precisa y personalizada.
3. Mantente amable, profesional y enfocado únicamente en temas relacionados con inversiones, finanzas y economía.<|user|>: ¿En qué aspectos son diferentes UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C?<|assistant|>: 
<|assistant|>
Las UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C son dos ETFs (Exchange Traded Funds) que se comercializan en Europa, pero presentan diferencias en sus objetivos de inversión.

La UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (E

## Guardado de todos los archivos
Por último, dado que Google Colab no permite descargar varios archivos o carpetas enteras, este bloque reune todos los archivos utilizados en el proyecto en un arhivo comprimido y lo descarga para poder reutilizarlo en futuras ocasiones.

Nota: la última linea (files.download("/content/file.zip")) descarga automaticamente el arhivo comprimido, pero con un tiempo de espera muy elevado. Para descargar el archivo más rápidamente una vez ya generado, podemos dirigirnos al panel de la zona izquierda, clickar en el icono de la carpeta, y con botón derecho encima del archivo llamado 'file.zip', hacer click en descargar, lo cual será mucho más rápido que esperar a que el comando prepare la descarga.

In [ ]:
!zip -r /content/file.zip /content
from google.colab import files
files.download("/content/file.zip")

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.config/default_configs.db (deflated 98%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/.last_update_check.json (deflated 22%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2024.12.19/ (stored 0%)
updating: content/.config/logs/2024.12.19/14.20.18.151587.log (deflated 58%)
updating: content/.config/logs/2024.12.19/14.20.29.520330.log (deflated 57%)
updating: content/.config/logs/2024.12.19/14.20.16.940511.log (deflated 87%)
updating: content/.config/logs/2024.12.19/14.19.43.316528.log (deflated 93%)
updating: content/.config/logs/2024.12.19/14.20.30.129972.log (deflated 57%)
updating: content/.config/logs/2024.12.19/14.20.05.781718.log (deflated 58%)
updating: content/.con

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>